# Assignment 1 (Due 25 January)

The goal of this assignment is to write a simple particle simulator. Assume we have $N$ particles with strength $k_j > 0$ at the positions $x_j\in\mathbb{R}^2$, $j=0,\dots, N-1$. Each particle creates a potential at a given point $x$ of strength $V_j(x) = -k_j\log |x-x_j|$. The total potential at a given position $x$ is therefore
$$
V(x) = -\sum_{j=0}^{N-1}k_j\log|x-x_j|.
$$

The following code generates $N$ particle positions and a vector of associated strengths.

In [ ]:
import numpy as np

N = 100

rand = np.random.RandomState(0)
pos = rand.rand(N, 2)
k = rand.rand(N)

In [ ]:
import pyopencl as cl
ctx = cl.create_some_context()
queue = cl.CommandQueue

Let us plot the corresponding particle positions.

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)
ax.plot(pos[:, 0], pos[:, 1], 'k.', markersize=4)
ax.set_aspect('equal')

Implement the following function **evaluate_field** that evaluates the potential field at a given set of points:

In [ ]:
def evaluate_field(evaluation_points, source_positions, strength_vector,
                   implementation='numba'):
    """
    Evaluate a field at a given set of points.
    
    Parameters
    ------------
    evaluation_points : np.ndarray
        A (M, 2) float64 array of M positions that contain the 
        two dimensional points at which to evaluate the field.
    source_positions : np.ndarray
        A (N, 2) float64 array of N source positions that contain
        the source locations
    strength_vector : np.ndarray
        A (N, ) float64 array of the strengths associated with
        each source.
    implementaiton : string
        Either 'numba' or 'opencl' to use a Numba or OpenCL
        implementation.
        
    Returns
    -------------
    A (M, ) float64 vector of the field values at the corresponding
    evaluation points.
    
    """
    if implementation == 'opencl':
        return evaluate_field_opencl(evaluation_points, source_positions,
                                     strength_vector)

def evaluate_field_opencl(evaluation_points, source_positions,
                          strength_vector):
    """OpenCL Implementation"""
    
    kernel = """
    
    __kernel void evaluate_potential(__global double *evaluationPoints,
                                     __global double *sourcePoints,
                                     __global double *strengthVector,
                                     __global double *result)
    {
    
    int gid = get_global_id(0);
    
    // Example assignment of result back
    
    double myResult = 0;
    
    result[gid] = myResult;
    
    // How to debug
    // The following prints out myResult on work item 0 using the
    // system stdout.
    
    if (gid == 0) printf(\"myResult\\n\", myResult);
        
    }
    
    """ 
    
    n_eval_points = evaluation_points.shape[0]
    n_source_positions = source_positions.shape[0]
    
    mf = cl.mem_flags
    
    eval_buffer = cl.Buffer(ctx, mf.COPY_HOST_PTR | mf.READ_ONLY, hostbuf=evaluation_points)
    source_buffer = cl.Buffer(ctx, mf.COPY_HOST_PTR | mf.READ_ONLY, hostbuf=source_positions)
    strength_buffer = cl.Buffer(ctx, mf.COPY_HOST_PTR | mf.READ_ONLY, hostbuf=strength_vector)
    result_buffer = cl.Buffer(ctx, mf.ALLOC_HOST_PTR, size=evaluation_points.shape[0] * 8)
    
    prg = cl.Program(ctx, kernel)
    prg.build()
    potential_kernel = prg.evaluate_potential
    
    potential_kernel(queue, (n_eval_points, ), (1, ),
                     eval_buffer, source_buffer, strength_buffer, result_buffer)
    
    result, _ = cl.enqueue_map_buffer(queue, result_buffer, cl.map_flags.READ,
                                      0, (n_eval_points, ), np.double)
    queue.finish()
    
    return result

To implement this function it makes sense not to write everything as one single function, but to have two implementations, **evaluate_field_opencl** and **evaluate_field_numba**, which are chosen depending on the *implementation* parameter.

To allow the marker to check the correctess of your implementation run the following test.

In [ ]:
evaluation_points = np.array([[1.5, 1.3], [1.8, 1.6]], dtype='float64')

result_numba = evaluate_field(evaluation_points, pos, k, implementation='numba')
result_opencl = evaluate_field(evaluation_points, pos, k, implementation='opencl')

print(result_numba)
print(result_opencl)

Obviously, both results should be identical. For OpenCL you should work with *float64* data types. The code must run using the pocl CPU OpenCL driver provided on Microsoft Azure Notebook.

Finally, we want to make timing measurements and also plot the results. Let us define a grid of evaluation points in the unit square.

In [ ]:
Nx = 100
Ny = 100
xpoints, ypoints = np.mgrid[0 : 1: 1j * Nx, 0 : 1: 1j * Ny]
evaluation_points = np.vstack([xpoints.ravel(), ypoints.ravel()]).T

We also need our usual timer class.

In [ ]:
import time

class Timer:    
    def __enter__(self):
        self.start = time.time()
        return self

    def __exit__(self, *args):
        self.end = time.time()
        self.interval = self.end - self.start

Now run the following two timing experiments:

In [ ]:
with Timer() as t:
    result_numba = evaluate_field(evaluation_points, pos, k, implementation='numba')
print(t.interval)

with Timer() as t:
    result_opencl = evaluate_field(evaluation_points, pos, k, implementation='opencl')
print(t.interval)

Finally, we want to plot the results. We plot the logs since the result will be large close to the poles.

In [ ]:
result_numba = np.log10(np.abs(result_numba.reshape(Nx, Ny)))
result_opencl = np.log10(np.abs(result_opencl.reshape(Nx, Ny)))

In [ ]:
figure = plt.figure(figsize=(20, 20))
ax1 = figure.add_subplot(121)
ax2 = figure.add_subplot(122)

ax1.imshow(result_numba.T, extent=[0, 1, 0, 1], origin='lower')
ax1.set_aspect('equal')
ax1.set_title('Numba')

ax2.imshow(result_opencl.T, extent=[0, 1, 0, 1], origin='lower')
ax2.set_aspect('equal')
ax2.set_title('OpenCL')


## Marking

A correct Numba implementation is worth 50%. A correct OpenCL implementation is worth a further 20%.
Coding style (clean code following PEP8 standard and good comments) is worth 10%.
Finally, I am comparing timing results from all implementations on my machine. The solutions from all students will be ranked and given bonus
marks depending on their ranking (20 bonus marks for the fastest solution, 0 bonus marks for the slowest solution). Bonus marks will only be
added for students who provide a correct Numba and OpenCL implementation.

**Make sure that your implementation of the evaluate_field function exactly follows the given signature. Changing the signature results in
0 marks being given for the assignment.**

To submit your assignment a corresponding submission link on Moodle will be enabled. Please submit one Jupyter notebook. Multi-file solutions will not be accepted.